## 18. Нормализация: 1NF

### 🧮 Нормализация: 1NF (Первая нормальная форма)

**Постановка проблемы:**  
Если хранить в одной ячейке список значений (например, все телефоны клиента через запятую), то будет трудно искать, фильтровать и связывать данные. Первая нормальная форма требует избавиться от повторяющихся групп.

**Краткая теория:**  
Таблица находится в **1NF**, если:
- Все поля атомарны (одно значение на ячейку)
- Нет повторяющихся столбцов
- Каждая строка уникальна

Для этого часто нужно **разделить таблицу** на несколько связанных: например, клиентов и их телефоны.

**Пример:**

In [ ]:
# Демонстрация 1NF в SQLite
import sqlite3

# Нарушение 1NF: телефоны через запятую
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE bad_clients (
    id INTEGER PRIMARY KEY,
    name TEXT,
    phones TEXT  -- нарушение: список телефонов через запятую
)''')
cursor.execute("INSERT INTO bad_clients (name, phones) VALUES (?, ?)", ("Анна", "+7900,+7901"))
cursor.execute("SELECT * FROM bad_clients")
print("🚫 Нарушение 1NF:")
for row in cursor.fetchall():
    print(row)

# Решение: выносим телефоны в отдельную таблицу
cursor.execute("DROP TABLE bad_clients")
cursor.execute("CREATE TABLE clients (id INTEGER PRIMARY KEY, name TEXT)")
cursor.execute("CREATE TABLE phones (
    id INTEGER PRIMARY KEY,
    client_id INTEGER,
    phone TEXT,
    FOREIGN KEY (client_id) REFERENCES clients(id)
)")
cursor.execute("INSERT INTO clients (name) VALUES ('Анна')")
client_id = cursor.lastrowid
cursor.executemany("INSERT INTO phones (client_id, phone) VALUES (?, ?)", [
    (client_id, "+7900"),
    (client_id, "+7901")
])
conn.commit()

# Проверим результат
print("\n✅ В нормальной форме:")
cursor.execute("SELECT c.name, p.phone FROM clients c JOIN phones p ON c.id = p.client_id")
for row in cursor.fetchall():
    print(row)